In [7]:
%pip install pdf2image


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [31]:
import pickle

from ollama import chat
from glob import glob
from tqdm import tqdm
from pdf2image import convert_from_path


In [ ]:
convert_from_path('./amazon-dynamo-sosp2007.pdf', output_folder='./images', fmt="jpeg")

[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1700x2200>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1700x2200>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1700x2200>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1700x2200>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1700x2200>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1700x2200>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1700x2200>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1700x2200>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1700x2200>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1700x2200>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1700x2200>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1700x2200>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1700x2200>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1700x2200>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=R

In [ ]:
response = chat(model='llama3.2-vision', messages=[
    {
        'role': 'user',
        'content': 'Generate metadata from these images of PDFs to help as a search retriver.',
        'images': ['./images/5bbab23d-c456-4392-b107-80b2ae5928b4-01.jpg']
    }
])
print(response.message.content)

The image shows a page from an academic paper titled "Dynamo: Amazon's Highly Available Key-Value Store" published in the Proceedings of the 2007 ACM SIGMOD International Conference on Management of Data. The paper discusses the design and implementation of Dynamo, a highly available key-value store developed by Amazon Web Services.

Here are the key points about the image:

* **Authors**: 
	+ Giuseppe DeCandia
	+ Deniz Hastorun
	+ Madan Jampani
	+ Gunavardhan Kakulapati
	+ Avinash Lakshman
	+ Alex Pilchin
	+ Swaminathan Sivasubramanian
	+ Peter Vosshall
	+ Werner Vogels

* **Abstract**:
	+ Reliability at massive scale is one of the biggest challenges faced by Amazon.com, with even minor outages having significant financial consequences.
	+ The paper presents the design and implementation of Dynamo, a highly available key-value store that provides an "always-on" experience.

* **Categories and Subject Descriptors**: 
	+ D.4.2 [Operating Systems]: Storage Management
	+ D.4.5 [Operating 

In [32]:
image_paths = glob('./images/*.jpg')

In [23]:
image_summaries = []

IMAGE_PAGE_EXTRACTION_PROMPT = """
    Extract summaries of the information from these images of PDFs to help a search retriver. 
    Create a summary of the content, sections headers in the document, 
    and information regarding images or charts in the document.
"""

for image_path in tqdm(image_paths):
    response = chat(model='llama3.2-vision', messages=[
        {
            'role': 'user',
            'content': IMAGE_PAGE_EXTRACTION_PROMPT,
            'images': [image_path]
        }
    ])
    extraction = response.message.content

    image_summaries.append({
        'image_id': image_path.split('/')[-1],
        'summary': extraction
    })

100%|██████████| 16/16 [24:03<00:00, 90.23s/it]


In [24]:
# TODO: Extract text from PDF and persist chunks.

In [26]:
image_summaries

[{'image_id': 'aef9fd24-f6ea-469a-9890-aa4d34cc50ec-15.jpg',
  'summary': 'The image displays a page from an academic paper, specifically the acknowledgments section. The purpose of this page is to acknowledge the contributions and support received by the authors during their research.\n\nHere are the key points about the content:\n\n* **Acknowledgments Section**: \n  * The acknowledgments section is located at the bottom left corner of the page.\n  * It contains a list of names, institutions, or organizations that have provided financial support, data, or other resources for the study.\n* **References Cited**:\n  * The references cited in this section are listed in alphabetical order by author\'s last name.\n  * Each reference is formatted according to a specific citation style (likely APA or MLA).\n* **Page Layout and Design**:\n  * The page has a clear header with the title of the paper, "Acknowledgements," centered at the top.\n  * The text is arranged in a single column, with head

In [29]:
with open('image_summaries.pkl', 'wb') as f:
    pickle.dump(image_summaries, f)

In [25]:
import chromadb

chroma_client = chromadb.Client()

ModuleNotFoundError: No module named 'chromadb'

In [ ]:
page_summary_collection = chroma_client.create_collection(name="page_summaries", metadata={"hnsw:space": "cosine"})